In [1]:
import numpy as np
import pandas as pd
import warnings
import time
from scipy import signal
from scipy.signal import butter, lfilter
import matplotlib.pyplot as plt
import math
import pickle
import statistics
import random
from sklearn.metrics.pairwise import cosine_similarity
from math import log
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import sktime as sktime
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sktime.datasets import load_from_tsfile_to_dataframe
from sktime.alignment.dtw_python import AlignerDTWfromDist
from sktime.dists_kernels.scipy_dist import ScipyDist
from sktime.dists_kernels.compose_from_align import DistFromAligner
import os
import sys
%matplotlib inline
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import RidgeClassifierCV
from sktime.transformations.panel.rocket import Rocket
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.classification.hybrid import HIVECOTEV2
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Dictionary saved to tb_hb_EMO_pain_dtwknn_normalized.pickle


### Load normal and abnormal datasets

In [2]:
final_normal_df = pd.read_pickle('normal_EMO_pain.pkl')
final_abnormal_df = pd.read_pickle('abnormal_df.pkl')

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,Subject
0,1.06100,0.98543,0.99366,0.97667,1.077800,1.02220,0.91566,0.91127,1.0622,0.99518,...,1.836900e-05,6.842100e-07,1.117200e-06,1.427500e-06,5.351800e-05,0.003015,0.001449,0.003680,0.103040,1
1,1.06110,0.98530,0.99360,0.97673,1.077800,1.02220,0.91565,0.91106,1.0627,0.99497,...,1.638800e-05,2.404500e-06,1.436100e-06,1.236500e-06,2.850300e-05,0.009904,0.002355,0.004888,0.058659,1
2,1.06120,0.98517,0.99356,0.97680,1.077800,1.02210,0.91564,0.91102,1.0631,0.99491,...,6.433300e-06,1.724400e-07,1.402800e-06,1.299000e-06,2.160700e-05,0.007058,0.002968,0.004549,0.028170,1
3,1.06130,0.98512,0.99351,0.97688,1.077800,1.02210,0.91558,0.91076,1.0633,0.99500,...,3.369700e-06,3.844300e-07,2.665200e-06,1.013900e-06,9.410500e-06,0.003072,0.001221,0.002663,0.012835,1
4,1.06130,0.98511,0.99348,0.97693,1.077800,1.02200,0.91566,0.91048,1.0635,0.99514,...,2.509200e-06,1.111800e-06,1.414900e-08,1.861300e-06,2.202100e-06,0.002561,0.000981,0.001777,0.005852,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204395,0.74410,0.81215,0.17334,0.33478,0.096342,0.14146,0.42687,0.91240,1.0285,1.02350,...,3.952500e-08,6.194100e-07,2.117300e-06,9.440300e-08,2.288800e-06,0.036068,0.142310,0.056785,0.062339,1323
204396,0.74429,0.81232,0.17336,0.33499,0.096483,0.14161,0.42610,0.90873,1.0286,1.02380,...,1.652200e-08,1.703000e-07,5.088200e-07,2.560500e-09,3.187700e-07,0.042179,0.122040,0.056728,0.015564,1323
204397,0.74460,0.81289,0.17296,0.33556,0.097070,0.14222,0.42865,0.89959,1.0288,1.02470,...,8.104400e-08,2.119300e-06,6.118100e-06,8.696300e-07,5.536500e-06,0.029771,0.136220,0.049647,0.044036,1323
204398,0.74474,0.81321,0.17264,0.33601,0.097671,0.14267,0.42972,0.89257,1.0288,1.02520,...,5.492400e-09,5.706100e-07,2.717700e-06,8.520200e-08,3.085600e-06,0.056131,0.059284,0.046159,0.053046,1323


,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,activity,subject_number,new_subject_number
0,1.02130,0.95783,0.98112,0.99898,1.067500,1.017300,0.90871,0.93430,1.0717,1.02430,...,1.361400e-05,3.357200e-07,7.035800e-03,0.002144,0.001760,0.002275,0.072891,1,11,1
1,1.01970,0.96150,0.97944,1.00050,1.066500,1.018100,0.90840,0.93432,1.0716,1.02450,...,1.490200e-05,4.935200e-06,7.383200e-03,0.002138,0.001701,0.001565,0.017368,1,11,1
2,1.01810,0.96536,0.97762,1.00210,1.065600,1.018600,0.90815,0.93394,1.0714,1.02510,...,1.412200e-05,5.239900e-06,8.312100e-03,0.002340,0.001532,0.001074,0.047489,1,11,1
3,1.01640,0.96948,0.97589,1.00370,1.064700,1.018800,0.90804,0.93396,1.0711,1.02560,...,1.076700e-05,9.999400e-06,8.300100e-03,0.002441,0.001697,0.001170,0.080234,1,11,1
4,1.01440,0.97376,0.97400,1.00530,1.063700,1.019000,0.90803,0.93410,1.0706,1.02620,...,7.268000e-06,1.286100e-05,7.484100e-03,0.002255,0.001039,0.001428,0.062073,1,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60195,0.74097,0.80707,0.18271,0.29593,0.077513,0.039578,0.19951,0.58894,1.0661,0.91215,...,1.058700e-06,1.934400e-05,4.672600e-06,0.024459,0.055559,0.067210,0.023681,2,1320,301
60196,0.74466,0.81115,0.18536,0.29860,0.082479,0.035784,0.20168,0.58315,1.0675,0.91468,...,8.426500e-07,1.742300e-05,3.997800e-06,0.136790,0.124810,0.022974,0.028399,2,1320,301
60197,0.74785,0.81456,0.18770,0.30148,0.087768,0.032743,0.20399,0.58189,1.0684,0.91865,...,7.763100e-07,1.639600e-05,4.348600e-06,0.088216,0.353800,0.049788,0.035921,2,1320,301
60198,0.75158,0.81844,0.19015,0.30415,0.092787,0.029651,0.20600,0.58223,1.0687,0.92150,...,8.127500e-07,1.261500e-05,1.290300e-06,0.023287,0.139760,0.017916,0.028992,2,1320,301


In [3]:
final_abnormal_df = final_abnormal_df.drop(columns=['subject_number','activity'])
final_abnormal_df.rename(columns={'new_subject_number': 'Subject'}, inplace=True)

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,Subject
0,1.02130,0.95783,0.98112,0.99898,1.067500,1.017300,0.90871,0.93430,1.0717,1.02430,...,1.348800e-07,0.000006,1.361400e-05,3.357200e-07,7.035800e-03,0.002144,0.001760,0.002275,0.072891,1
1,1.01970,0.96150,0.97944,1.00050,1.066500,1.018100,0.90840,0.93432,1.0716,1.02450,...,3.186200e-07,0.000004,1.490200e-05,4.935200e-06,7.383200e-03,0.002138,0.001701,0.001565,0.017368,1
2,1.01810,0.96536,0.97762,1.00210,1.065600,1.018600,0.90815,0.93394,1.0714,1.02510,...,2.305600e-06,0.000015,1.412200e-05,5.239900e-06,8.312100e-03,0.002340,0.001532,0.001074,0.047489,1
3,1.01640,0.96948,0.97589,1.00370,1.064700,1.018800,0.90804,0.93396,1.0711,1.02560,...,6.841300e-06,0.000017,1.076700e-05,9.999400e-06,8.300100e-03,0.002441,0.001697,0.001170,0.080234,1
4,1.01440,0.97376,0.97400,1.00530,1.063700,1.019000,0.90803,0.93410,1.0706,1.02620,...,1.340100e-05,0.000021,7.268000e-06,1.286100e-05,7.484100e-03,0.002255,0.001039,0.001428,0.062073,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60195,0.74097,0.80707,0.18271,0.29593,0.077513,0.039578,0.19951,0.58894,1.0661,0.91215,...,5.125400e-06,0.000022,1.058700e-06,1.934400e-05,4.672600e-06,0.024459,0.055559,0.067210,0.023681,301
60196,0.74466,0.81115,0.18536,0.29860,0.082479,0.035784,0.20168,0.58315,1.0675,0.91468,...,4.545600e-06,0.000014,8.426500e-07,1.742300e-05,3.997800e-06,0.136790,0.124810,0.022974,0.028399,301
60197,0.74785,0.81456,0.18770,0.30148,0.087768,0.032743,0.20399,0.58189,1.0684,0.91865,...,1.740600e-06,0.000035,7.763100e-07,1.639600e-05,4.348600e-06,0.088216,0.353800,0.049788,0.035921,301
60198,0.75158,0.81844,0.19015,0.30415,0.092787,0.029651,0.20600,0.58223,1.0687,0.92150,...,1.877700e-07,0.000018,8.127500e-07,1.261500e-05,1.290300e-06,0.023287,0.139760,0.017916,0.028992,301


In [4]:
abnormal_indices = (final_abnormal_df['Subject']).unique()
abnormal_no = (final_abnormal_df['Subject']).nunique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

301

In [5]:
# Specify the file path of the saved pkl file
file_path = 'final_bins_67.pkl'

# Load the dictionary from the pkl file
with open(file_path, 'rb') as file:
    final_numbers_dictionary = pickle.load(file)
# Now, 'loaded_dictionary' contains the dictionary loaded from the pkl file
display("Dictionary loaded from pickle file:")

'Dictionary loaded from pickle file:'

{'sample_numbers_list_0.0_to_0.1': [145,
  52,
  9,
  80,
  102,
  106,
  116,
  146,
  159,
  166,
  197,
  258,
  292,
  305,
  308,
  311,
  314,
  315,
  317,
  323,
  348,
  352,
  354,
  370,
  373,
  376,
  421,
  432,
  436,
  438,
  449,
  458,
  543,
  559,
  567,
  637,
  666,
  692,
  693,
  718,
  722,
  753,
  762,
  802,
  807,
  812,
  883,
  898,
  931,
  954,
  959,
  994,
  1016,
  1027,
  1034,
  1074,
  1083,
  1094,
  1096,
  1118,
  1121,
  1128,
  1131,
  1133,
  1136,
  1148,
  1149,
  1262,
  1281,
  1317,
  1321],
 'sample_numbers_list_0.1_to_0.2': [686,
  676,
  268,
  64,
  76,
  127,
  156,
  201,
  207,
  232,
  241,
  253,
  257,
  272,
  276,
  285,
  306,
  310,
  342,
  386,
  427,
  439,
  451,
  454,
  474,
  483,
  490,
  542,
  607,
  616,
  638,
  644,
  647,
  661,
  765,
  767,
  793,
  803,
  1018,
  1049,
  1093,
  1099,
  1114,
  1115,
  1138,
  1146,
  1179,
  1188,
  1190,
  1210,
  1214,
  1286,
  1287,
  1292,
  1316,
  1323],
 'sample_n

### Function to find the length of the time series

In [6]:
# Define a function to z-normalize a column
def z_normalize(column):
    mean = column.mean()
    std = column.std()
    z_normalized = (column - mean) / std
    return z_normalized

In [7]:
def time_series_length_finding(df):
    train_1_abnormal = df.copy()
    train_1_ab_sub = (train_1_abnormal['Subject'].unique()).tolist()
    row_count = []
    for i in train_1_ab_sub:
        train_1_abnormal_new = train_1_abnormal[(train_1_abnormal['Subject']== i)]
        no_rows = train_1_abnormal_new.shape[0]
        row_count.append(no_rows)
    display("Avg length:",statistics.mean(row_count))
    avg_len_ts = int(statistics.mean(row_count))
    return(avg_len_ts)

In [8]:
def preprocessing_normal_df(df,desired_length):
    train_2_normal = df.copy()
    df_train_3 = pd.DataFrame(columns = train_2_normal.columns)
    num_dim = df_train_3.shape[1]
    display(num_dim)
    df_train_3.columns = range(len(df_train_3.columns))
    train_2_sub = (train_2_normal['Subject'].unique()).tolist()
    
    m = 0
    for i in train_2_sub:
        train_2_normal_new = train_2_normal[(train_2_normal['Subject']== i)]
        df_row_count = train_2_normal_new.shape[0]
        if df_row_count < desired_length:
            rows_to_add = desired_length - df_row_count
            train_2_normal_new.tail()
            last_row = train_2_normal_new.ffill().iloc[[-1]]  # Extract the last row
            new_rows_df = pd.concat([last_row] * rows_to_add, ignore_index=True)
            train_2_normal_initial = pd.concat([train_2_normal_new, new_rows_df], ignore_index=True)
            train_2_normal_initial_no_sub = train_2_normal_initial.drop(columns='Subject')
            train_2_normal_final = z_normalize(train_2_normal_initial_no_sub)
            train_2_normal_final.insert(0, 'Subject', i)
        else:
            train_2_normal_initial = train_2_normal_new.iloc[:desired_length,:]
            train_2_normal_initial_no_sub = train_2_normal_initial.drop(columns='Subject')
            train_2_normal_final = z_normalize(train_2_normal_initial_no_sub)
            train_2_normal_final.insert(0, 'Subject', i)
            
        train_2_normal_final.columns = range(len(train_2_normal_final.columns))
        
        df_train_3.at[m,0] = i
        j = 0
        while j < num_dim:
            df_train_3.at[m,j] = train_2_normal_final[j]
            j += 1
        m += 1    

    df_train_3['class'] = 'normal'
    df_normal_baseline = df_train_3.iloc[:,1:]
    return(df_normal_baseline)

In [9]:
def preprocessing_dataframe(df,desired_length):
    train_1_abnormal = df.copy()
    df_train_2 = pd.DataFrame(columns = train_1_abnormal.columns)
    num_dim_ab = df_train_2.shape[1]

    df_train_2.columns = range(len(df_train_2.columns))
 
    train_1_ab_sub = (train_1_abnormal['Subject'].unique()).tolist()
   
    m = 0
    for i in train_1_ab_sub:
        train_1_abnormal_new = train_1_abnormal[(train_1_abnormal['Subject']== i)]
        df_row_count = train_1_abnormal_new.shape[0]
        if df_row_count < desired_length:
            rows_to_add = desired_length - df_row_count
            
            last_row = train_1_abnormal_new.ffill().iloc[[-1]]  # Extract the last row
            new_rows_df = pd.concat([last_row] * rows_to_add, ignore_index=True)
            train_1_abnormal_initial = pd.concat([train_1_abnormal_new, new_rows_df], ignore_index=True)
            train_1_abnormal_initial_no_sub = train_1_abnormal_initial.drop(columns='Subject')
            train_1_abnormal_final = z_normalize(train_1_abnormal_initial_no_sub)
            train_1_abnormal_final.insert(0, 'Subject', i)
        else:
            train_1_abnormal_initial = train_1_abnormal_new.iloc[:desired_length,:]
            train_1_abnormal_initial_no_sub = train_1_abnormal_initial.drop(columns='Subject')
            train_1_abnormal_final = z_normalize(train_1_abnormal_initial_no_sub)
            train_1_abnormal_final.insert(0, 'Subject', i)
        train_1_abnormal_final.columns = range(len(train_1_abnormal_final.columns))
        
        df_train_2.at[m,0] = i
        j = 0
        while j < num_dim_ab:
            df_train_2.at[m,j] = train_1_abnormal_final[j]
            j += 1
        m += 1    
    
    df_train_2['class'] = 'abnormal'
    df_final = df_train_2.iloc[:,1:]
    return(df_final)

In [10]:
def ts_classifier(df_class_1):
    accuacy_list_1 = []
    precision_list_1 = []
    recall_list_1 = []
    f1_list_1 = []

    X_1 = df_class_1.iloc[:,:-1]
    y_1 = df_class_1.iloc[:,-1]
    
    display("X_1:",X_1)

    for i in range(1,31):
        X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1, test_size=0.2, random_state=i,stratify = y_1)

        dtw_knn = KNeighborsTimeSeriesClassifier(distance="dtw")
        dtw_knn.fit(X_train_1, y_train_1)
        y_pred_1 = dtw_knn.predict(X_test_1)

        pos_label = 'abnormal'

        acc_1 = accuracy_score(y_test_1, y_pred_1)
        accuacy_list_1.append(acc_1)

        prec_1 = precision_score(y_test_1, y_pred_1, pos_label=pos_label)
        precision_list_1.append(prec_1)

        recall_1 = recall_score(y_test_1, y_pred_1, pos_label=pos_label)
        recall_list_1.append(recall_1)

        f1_1 = f1_score(y_test_1, y_pred_1, pos_label=pos_label)
        f1_list_1.append(f1_1)
        
        display("Completed",i)

    acc_mean_1 = np.mean(accuacy_list_1)
    acc_std_dev_1 = np.std(accuacy_list_1)

    prec_mean_1 = np.mean(precision_list_1)
    prec_std_dev_1 = np.std(precision_list_1)

    recall_mean_1 = np.mean(recall_list_1)
    recall_std_dev_1 = np.std(recall_list_1)

    f1_mean_1 = np.mean(f1_list_1)
    f1_std_dev_1 = np.std(f1_list_1)
    
    return(acc_mean_1,acc_std_dev_1,prec_mean_1,prec_std_dev_1,recall_mean_1,recall_std_dev_1,f1_mean_1,f1_std_dev_1)

In [11]:
# Create a new dictionary with combined values
# Keys to combine
key_to_combine1 = 'sample_numbers_list_0.0_to_0.1'
key_to_combine2 = 'sample_numbers_list_0.1_to_0.2'

# Create a new dictionary with combined values
hb_dict = {'hb_key': final_numbers_dictionary[key_to_combine1] + final_numbers_dictionary[key_to_combine2]}

{'hb_key': [145,
  52,
  9,
  80,
  102,
  106,
  116,
  146,
  159,
  166,
  197,
  258,
  292,
  305,
  308,
  311,
  314,
  315,
  317,
  323,
  348,
  352,
  354,
  370,
  373,
  376,
  421,
  432,
  436,
  438,
  449,
  458,
  543,
  559,
  567,
  637,
  666,
  692,
  693,
  718,
  722,
  753,
  762,
  802,
  807,
  812,
  883,
  898,
  931,
  954,
  959,
  994,
  1016,
  1027,
  1034,
  1074,
  1083,
  1094,
  1096,
  1118,
  1121,
  1128,
  1131,
  1133,
  1136,
  1148,
  1149,
  1262,
  1281,
  1317,
  1321,
  686,
  676,
  268,
  64,
  76,
  127,
  156,
  201,
  207,
  232,
  241,
  253,
  257,
  272,
  276,
  285,
  306,
  310,
  342,
  386,
  427,
  439,
  451,
  454,
  474,
  483,
  490,
  542,
  607,
  616,
  638,
  644,
  647,
  661,
  765,
  767,
  793,
  803,
  1018,
  1049,
  1093,
  1099,
  1114,
  1115,
  1138,
  1146,
  1179,
  1188,
  1190,
  1210,
  1214,
  1286,
  1287,
  1292,
  1316,
  1323]}

In [12]:
classifier_dict_hb = {}
key = 'hb_key'
normal_sample_numbers = [] 
normal_sample_numbers.extend(hb_dict[key])
normal_sample_length = len(normal_sample_numbers)
display(normal_sample_length)

if len(normal_sample_numbers) <= abnormal_no:
    df_normal = final_normal_df[final_normal_df['Subject'].isin(normal_sample_numbers)]
    display(df_normal.head())

    random.seed(42)
    abnormal_sample_numbers = random.sample(abnormal_indices.tolist(), normal_sample_length)
    df_abnormal = final_abnormal_df[final_abnormal_df['Subject'].isin(abnormal_sample_numbers)]
    display(df_abnormal.head())

    desired_length_for_ts = time_series_length_finding(df_abnormal)

    df_normal_preprocessed = preprocessing_normal_df(df_normal,desired_length_for_ts)
    df_abnormal_preprocessed = preprocessing_dataframe(df_abnormal,desired_length_for_ts)

    df_classifier = (pd.concat([df_normal_preprocessed,df_abnormal_preprocessed]).reset_index()).drop(columns='index')
    acc_mean,acc_std_dev,prec_mean,prec_std_dev,recall_mean,recall_std_dev,f1_mean,f1_std_dev = ts_classifier(df_classifier)

    classifier_dict_hb[key] = {'acc_mean': acc_mean,'acc_std_dev': acc_std_dev,'prec_mean': prec_mean,'prec_std_dev': prec_std_dev,'recall_mean': recall_mean,'recall_std_dev': recall_std_dev,'f1_mean': f1_mean,'f1_std_dev': f1_std_dev}

else:
    df_abnormal = final_abnormal_df

    random.seed(42)
    random_sample_numbers = random.sample(normal_sample_numbers, abnormal_no)
    df_normal = final_normal_df[final_normal_df['Subject'].isin(random_sample_numbers)]
    display(df_normal)

    desired_length_for_ts = time_series_length_finding(df_abnormal)

    df_normal_preprocessed = preprocessing_normal_df(df_normal,desired_length_for_ts)
    df_abnormal_preprocessed = preprocessing_dataframe(df_abnormal,desired_length_for_ts)

    df_classifier = (pd.concat([df_normal_preprocessed,df_abnormal_preprocessed]).reset_index()).drop(columns='index')
    acc_mean,acc_std_dev,prec_mean,prec_std_dev,recall_mean,recall_std_dev,f1_mean,f1_std_dev = ts_classifier(df_classifier)

    classifier_dict_hb[key] = {'acc_mean': acc_mean,'acc_std_dev': acc_std_dev,'prec_mean': prec_mean,'prec_std_dev': prec_std_dev,'recall_mean': recall_mean,'recall_std_dev': recall_std_dev,'f1_mean': f1_mean,'f1_std_dev': f1_std_dev}

127

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,Subject
1600,1.0643,0.97671,1.0070,0.97521,1.0703,1.0165,0.90941,0.94086,1.0815,0.99308,...,1.390000e-07,1.148600e-06,1.046100e-06,4.121200e-07,3.225500e-08,0.000745,0.000475,0.001632,0.028687,9
1601,1.0644,0.97647,1.0071,0.97521,1.0703,1.0167,0.90961,0.94119,1.0815,0.99321,...,9.799100e-08,9.781300e-07,2.860700e-08,3.829800e-07,3.087800e-06,0.000971,0.000594,0.002522,0.033865,9
1602,1.0644,0.97623,1.0071,0.97524,1.0702,1.0169,0.90969,0.94160,1.0815,0.99317,...,4.181300e-08,8.681200e-08,1.591600e-07,4.911800e-11,6.580000e-06,0.001313,0.001621,0.002045,0.043725,9
1603,1.0645,0.97588,1.0072,0.97529,1.0701,1.0174,0.90974,0.94143,1.0816,0.99326,...,3.735300e-07,4.374800e-07,5.836400e-10,8.244200e-07,1.464900e-05,0.002589,0.001037,0.004383,0.024257,9
1604,1.0646,0.97558,1.0074,0.97529,1.0699,1.0178,0.90981,0.94151,1.0817,0.99313,...,8.462700e-07,9.562100e-07,2.120300e-08,9.132500e-07,3.586100e-05,0.002062,0.000424,0.002832,0.011480,9


,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,Subject
600,0.98664,0.98955,0.92570,1.0152,0.99745,1.0258,0.91296,0.90637,1.0585,1.0150,...,1.891600e-07,7.314500e-06,2.348800e-07,2.402500e-06,0.000338,0.001444,0.001632,0.000884,0.004008,4
601,0.98464,0.99143,0.92362,1.0169,0.99102,1.0260,0.91191,0.90626,1.0586,1.0151,...,1.490700e-06,1.345100e-07,1.148100e-05,4.361600e-07,0.000305,0.003219,0.001500,0.001381,0.003962,4
602,0.98254,0.99326,0.92158,1.0184,0.98427,1.0258,0.91052,0.90664,1.0588,1.0153,...,1.863700e-06,1.510000e-06,1.045100e-05,4.042000e-08,0.000235,0.003939,0.002752,0.000854,0.001827,4
603,0.98022,0.99502,0.91938,1.0199,0.97713,1.0258,0.90882,0.90702,1.0588,1.0154,...,8.563500e-08,8.720400e-07,1.081100e-05,3.521900e-06,0.000154,0.003631,0.001928,0.000773,0.002756,4
604,0.97773,0.99673,0.91688,1.0217,0.97010,1.0259,0.90681,0.90746,1.0587,1.0154,...,1.687000e-06,3.935400e-08,1.632000e-05,1.174400e-05,0.000096,0.004376,0.002406,0.001088,0.004217,4


'Avg length:'

200

...

'accuracy: 0.705882'

'f1_score: 0.745763'

'Completed'

30

'F1 Mean: 0.745937'

In [13]:
import pickle

# Specify the file path where you want to save the pickle file
pickle_file_path = 'tb_hb_EMO_pain_dtwknn_normalized.pickle'

# Save the dictionary to a pickle file
with open(pickle_file_path, 'wb') as file:
    pickle.dump(classifier_dict_hb, file)

print(f"Dictionary saved to {pickle_file_path}")

In [14]:
classifier_dict_hb

{'hb_key': {'acc_mean': 0.727450980392157,
  'acc_std_dev': 0.057132557977778296,
  'prec_mean': 0.6950049466167978,
  'prec_std_dev': 0.05613682871298497,
  'recall_mean': 0.8103076923076924,
  'recall_std_dev': 0.08868635868589109,
  'f1_mean': 0.745936884256301,
  'f1_std_dev': 0.056957940657342825}}

In [16]:
all_values = [value for sublist in final_numbers_dictionary.values() for value in sublist]
healthy_dict = {'healthy_key': all_values}
display(healthy_dict)

{'healthy_key': [145,
  52,
  9,
  80,
  102,
  106,
  116,
  146,
  159,
  166,
  197,
  258,
  292,
  305,
  308,
  311,
  314,
  315,
  317,
  323,
  348,
  352,
  354,
  370,
  373,
  376,
  421,
  432,
  436,
  438,
  449,
  458,
  543,
  559,
  567,
  637,
  666,
  692,
  693,
  718,
  722,
  753,
  762,
  802,
  807,
  812,
  883,
  898,
  931,
  954,
  959,
  994,
  1016,
  1027,
  1034,
  1074,
  1083,
  1094,
  1096,
  1118,
  1121,
  1128,
  1131,
  1133,
  1136,
  1148,
  1149,
  1262,
  1281,
  1317,
  1321,
  686,
  676,
  268,
  64,
  76,
  127,
  156,
  201,
  207,
  232,
  241,
  253,
  257,
  272,
  276,
  285,
  306,
  310,
  342,
  386,
  427,
  439,
  451,
  454,
  474,
  483,
  490,
  542,
  607,
  616,
  638,
  644,
  647,
  661,
  765,
  767,
  793,
  803,
  1018,
  1049,
  1093,
  1099,
  1114,
  1115,
  1138,
  1146,
  1179,
  1188,
  1190,
  1210,
  1214,
  1286,
  1287,
  1292,
  1316,
  1323,
  167,
  1052,
  980,
  907,
  477,
  1019,
  1006,
  605,
  27,


In [17]:
classifier_dict_healthy = {}
key = 'healthy_key'
normal_sample_numbers = [] 
normal_sample_numbers.extend(healthy_dict[key])
normal_sample_length = len(normal_sample_numbers)
display(normal_sample_length)

if len(normal_sample_numbers) <= abnormal_no:
    df_normal = final_normal_df[final_normal_df['Subject'].isin(normal_sample_numbers)]
    display(df_normal.head())

    random.seed(42)
    abnormal_sample_numbers = random.sample(abnormal_indices.tolist(), normal_sample_length)
    df_abnormal = final_abnormal_df[final_abnormal_df['Subject'].isin(abnormal_sample_numbers)]
    display(df_abnormal.head())

    desired_length_for_ts = time_series_length_finding(df_abnormal)

    df_normal_preprocessed = preprocessing_normal_df(df_normal,desired_length_for_ts)
    df_abnormal_preprocessed = preprocessing_dataframe(df_abnormal,desired_length_for_ts)

    df_classifier = (pd.concat([df_normal_preprocessed,df_abnormal_preprocessed]).reset_index()).drop(columns='index')
    acc_mean,acc_std_dev,prec_mean,prec_std_dev,recall_mean,recall_std_dev,f1_mean,f1_std_dev = ts_classifier(df_classifier)

    classifier_dict_healthy[key] = {'acc_mean': acc_mean,'acc_std_dev': acc_std_dev,'prec_mean': prec_mean,'prec_std_dev': prec_std_dev,'recall_mean': recall_mean,'recall_std_dev': recall_std_dev,'f1_mean': f1_mean,'f1_std_dev': f1_std_dev}
    display(f1_mean)

else:
    df_abnormal = final_abnormal_df

    random.seed(42)
    random_sample_numbers = random.sample(normal_sample_numbers, abnormal_no)
    df_normal = final_normal_df[final_normal_df['Subject'].isin(random_sample_numbers)]
    display(df_normal)

    desired_length_for_ts = time_series_length_finding(df_abnormal)

    df_normal_preprocessed = preprocessing_normal_df(df_normal,desired_length_for_ts)
    df_abnormal_preprocessed = preprocessing_dataframe(df_abnormal,desired_length_for_ts)

    df_classifier = (pd.concat([df_normal_preprocessed,df_abnormal_preprocessed]).reset_index()).drop(columns='index')
    acc_mean,acc_std_dev,prec_mean,prec_std_dev,recall_mean,recall_std_dev,f1_mean,f1_std_dev = ts_classifier(df_classifier)
    display(f1_mean)

    classifier_dict_healthy[key] = {'acc_mean': acc_mean,'acc_std_dev': acc_std_dev,'prec_mean': prec_mean,'prec_std_dev': prec_std_dev,'recall_mean': recall_mean,'recall_std_dev': recall_std_dev,'f1_mean': f1_mean,'f1_std_dev': f1_std_dev}

820

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,Subject
200,1.03620,1.02450,1.00120,0.98164,1.067500,1.02820,0.91335,0.91552,1.0671,1.0089,...,2.427400e-06,7.043000e-06,5.066700e-05,7.993900e-07,2.353500e-03,0.002814,0.001536,0.001193,0.014422,2
201,1.03520,1.02530,1.00120,0.98167,1.067500,1.02810,0.91323,0.91584,1.0668,1.0094,...,4.998400e-06,1.333000e-05,4.240300e-05,1.047300e-06,2.368200e-03,0.002881,0.001030,0.000766,0.023694,2
202,1.03430,1.02620,1.00130,0.98169,1.067500,1.02800,0.91299,0.91610,1.0663,1.0101,...,1.367600e-05,2.731100e-05,3.289900e-05,9.528000e-07,2.370200e-03,0.001205,0.000703,0.002212,0.007037,2
203,1.03340,1.02690,1.00130,0.98175,1.067500,1.02790,0.91278,0.91634,1.0658,1.0107,...,1.373100e-05,1.570400e-05,2.975500e-05,1.202000e-06,2.139200e-03,0.001176,0.000621,0.001806,0.008661,2
204,1.03250,1.02760,1.00130,0.98183,1.067500,1.02770,0.91261,0.91656,1.0654,1.0111,...,8.030400e-06,9.615000e-06,2.327000e-05,1.900200e-06,2.049500e-03,0.000730,0.000501,0.009032,0.014776,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204195,0.73873,0.79443,0.16666,0.31097,0.082739,0.13984,0.63916,0.83980,1.0516,1.0093,...,2.113600e-07,4.046000e-07,2.527900e-07,1.614800e-08,3.570400e-07,0.125180,0.131890,0.024886,0.062635,1322
204196,0.73873,0.79463,0.16678,0.31119,0.082727,0.13987,0.63981,0.84018,1.0508,1.0100,...,1.293500e-06,1.072500e-06,3.468400e-08,1.197300e-08,6.631900e-08,0.152540,0.136790,0.008978,0.020723,1322
204197,0.73883,0.79490,0.16706,0.31147,0.082715,0.13976,0.64060,0.84116,1.0501,1.0105,...,1.052800e-06,7.577900e-07,1.771800e-08,3.139300e-08,1.021100e-08,0.040794,0.085858,0.024858,0.007888,1322
204198,0.73886,0.79505,0.16705,0.31138,0.082690,0.13956,0.64120,0.84175,1.0495,1.0109,...,7.707000e-07,3.205100e-07,2.461000e-08,2.333300e-08,9.462200e-08,0.107290,0.027815,0.035594,0.014827,1322


'Avg length:'

200

31

...

'f1_score: 0.727273'

'Completed'

30

'F1 Mean: 0.737801'

0.7378012917408215

In [18]:
classifier_dict_healthy

{'healthy_key': {'acc_mean': 0.6944903581267217,
  'acc_std_dev': 0.02437510647738142,
  'prec_mean': 0.6452653464982941,
  'prec_std_dev': 0.021019124228528183,
  'recall_mean': 0.8624499089253188,
  'recall_std_dev': 0.03566333924604666,
  'f1_mean': 0.7378012917408215,
  'f1_std_dev': 0.020855788255416625}}

In [19]:
import pickle

# Specify the file path where you want to save the pickle file
pickle_file_path = 'healthy_EMO_pain_dtwknn_normalized.pickle'

# Save the dictionary to a pickle file
with open(pickle_file_path, 'wb') as file:
    pickle.dump(classifier_dict_healthy, file)

print(f"Dictionary saved to {pickle_file_path}")